In [ ]:
! pip install implicit
! pip install transformers

# Clean Data

In [ ]:
import pandas as pd, numpy as np
import gzip
import json
import random
import ast

In [ ]:
def parse(path):
  """
  Parses a gzip file containing JSON objects, yielding each object.
  """
  
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path, sorted_indices):
  """
  Reads JSON objects from a file at sorted indices and creates a DataFrame.

  :param path: The path to the file containing JSON objects.
  :param sorted_indices: A list of indices specifying the desired objects to retrieve.

  :return: A Pandas DataFrame containing the selected JSON objects.
  """

  df = {}
  i = 0
  j = 0
  for d in parse(path):
      if i == sorted_indices[j]:
          df[i] = d
          j += 1
      i += 1
      # Optional: Break if all needed rows are read
      if len(df) == len(sorted_indices):
          break

      if i % 1000000 == 0:
        print(i // 1000000)
  return pd.DataFrame.from_dict(df, orient='index')


In [ ]:
random_indices = set(random.sample(range(41130001), 2000000))
sorted_indices = sorted(list(random_indices))

In [ ]:
df = getDF('data/kcore_5.json.gz',sorted_indices)
# Count the occurrences of each 'asin'
asin_counts = df['asin'].value_counts()

# Filter the counts to find asins that appear at least 5 times
asins_to_keep = asin_counts[asin_counts >= 5].index

# Filter the original DataFrame to only include those asins
df = df[df['asin'].isin(asins_to_keep)]
len(df['reviewerID'].unique())
df = df.drop(['reviewTime', 'helpful'], axis=1)
df

In [ ]:
data = {}

with gzip.open('data/metadata.json.gz', 'rb') as file:
    for i, line in enumerate(file):

        data[i]= eval(line.decode('utf-8'))

md = pd.DataFrame.from_dict(data, orient='index')
mddrop = md.drop(columns=['salesRank', 'description', 'related', 'brand'])
mddrop

In [ ]:
merged_df = pd.merge(df, mddrop, on='asin',how='left')
merged_df['review_length'] = merged_df['reviewText'].apply(len)
df = merged_df

In [ ]:
df.shape

(3060220, 12)

In [ ]:

def parse_cat_to_set(categories_string):
  """
  Parses a string representing nested categories and returns unique elements as a set.
  """
  
  if pd.isnull(categories_string):
    return set()

  categories = ast.literal_eval(categories_string)

  unique_elements = set()

  for category in categories:
      unique_elements.update(category)

  return unique_elements


In [ ]:
df['cat'] = df['categories'].apply(parse_cat_to_set)

In [ ]:
df = df.drop(columns=['categories','review_length','reviewerName'])

## Sentiment Analysis

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True, truncation = True, device=0)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
def get_unexpectedness(text):
  """
  Calculates the unexpectedness score for a single text using the Hugging Face pipeline function
  """
  output = classifier(text)

  # Return the unexpectedness score
  return output[0][6]['score']

def get_batch_unexpectedness(text_arr):
  """
  Calculate the unexpectedness score for an array of texts
  """
  outputs = classifier(text_arr)

  # Return the unexpectedness score
  result = [output[6]['score'] for output in outputs]

  return result

In [ ]:
# Replace nan value with empty string
text_lst = list(df['reviewText'])
for i,t in enumerate(text_lst):
  if type(t) is not str:
    text_lst[i] = ""

In [ ]:
df['unexpectedness'] = get_batch_unexpectedness(text_lst)

In [ ]:
df.to_csv('data/cleaned_df.csv',index=False)